In this practice code, we are going to use Ozone dataset. Dictionary of this dataset can be found in https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/airquality.html

In [38]:
import pandas as pd
import numpy as np

In [39]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)

#### Explore the dataset and decide which variables suffer from missing data

In [40]:
OzoneData.count()

Ozone      116
Solar.R    146
Wind       153
Temp       153
Month      153
Day        153
dtype: int64

Answer: 

#### Let's drop rows that have missing values in all the columns you indentified above

Hint: in dropna() if you set how = 'all', it will only drop columns that are suffering from missing values at all varaibles you introduce in subset. If you want to get rid of the row that contains missing values in any of the variables you specify, then you shall set how = 'any'

df.dropna(how = 'all',subset = ['Var1','Var2','Var3'],inplace = True)

The above code will check if all 3 variables specified in df have missing values, if they all have missing values it will drop that row.

In [41]:
OzoneData.dropna(how = 'all',subset = ['Solar.R','Ozone'],inplace = True)

In [42]:
OzoneData.count()

Ozone      116
Solar.R    146
Wind       151
Temp       151
Month      151
Day        151
dtype: int64

#### Spoiler! If everything is going according to plan you shall be left by 151 observations. Also, it seemed like the first two varibales had missing values. Now plese make a copy of your dataframe into a dataframe named OzoneImputeMean. Also, please use mean of the variables to fill in missing values in OzoneImputeMean

In [43]:
OzoneImputeMean = OzoneData.copy()
OzoneImputeMean['Ozone'].fillna(value = np.mean(OzoneImputeMean['Ozone']), inplace = True)
OzoneImputeMean['Solar.R'].fillna(value = np.mean(OzoneImputeMean['Solar.R']), inplace = True)

In [44]:
#Check if missing values are filled

OzoneImputeMean.count()

Ozone      151
Solar.R    151
Wind       151
Temp       151
Month      151
Day        151
dtype: int64

#### Now it's time for imputing using linear regression lines

In [45]:
#### Before we start let's define dummy variables for variable Month - don't worry about day!
MonthDummy = pd.get_dummies(OzoneData.Month, prefix = 'Month')
del MonthDummy['Month_9']
OzoneData = pd.concat([OzoneData, MonthDummy], axis=1)
OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,41,190,7.4,67,5,1,1,0,0,0
1,36,118,8.0,72,5,2,1,0,0,0
2,12,149,12.6,74,5,3,1,0,0,0
3,18,313,11.5,62,5,4,1,0,0,0
5,28,NaN,14.9,66,5,6,1,0,0,0


In [46]:
OzoneData.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.348342,-0.601547,0.698360,0.164519,-0.013226,-0.302963,-0.112006,0.277961,0.291808
Solar.R,0.348342,1.000000,-0.056792,0.275840,-0.075301,-0.150275,-0.024601,0.023998,0.176744,-0.076390
Wind,-0.601547,-0.056792,1.000000,-0.466032,-0.175317,0.042365,0.237781,0.046054,-0.144638,-0.166105
Temp,0.698360,0.275840,-0.466032,1.000000,0.397427,-0.136876,-0.637816,0.050753,0.318103,0.321680
Month,0.164519,-0.075301,-0.175317,0.397427,1.000000,-0.007727,-0.702257,-0.362131,-0.007201,0.355246
Day,-0.013226,-0.150275,0.042365,-0.136876,-0.007727,1.000000,0.011003,-0.017044,0.011471,0.011471
Month_5,-0.302963,-0.024601,0.237781,-0.637816,-0.702257,0.011003,1.000000,-0.242766,-0.247805,-0.247805
Month_6,-0.112006,0.023998,0.046054,0.050753,-0.362131,-0.017044,-0.242766,1.000000,-0.253080,-0.253080
Month_7,0.277961,0.176744,-0.144638,0.318103,-0.007201,0.011471,-0.247805,-0.253080,1.000000,-0.258333
Month_8,0.291808,-0.076390,-0.166105,0.321680,0.355246,0.011471,-0.247805,-0.253080,-0.258333,1.000000


#### What seems to be the list of best variables can define Ozone? how about Solar.R?

Answer: Temp, Wind, Solar.R, Month_5 ...  ; Ozone, Temp

In [47]:
# now let's use a regression model to predict Ozone. First drop NaN values in Ozone and save it in OzoneDroppedValues_Ozone
# the run a regression line on variables of interest and check significancy of your model
# if in a multi class dummy variable case, you see a few of the dummy variables are not significant but the rest are, 
#you shall either drop all or keep all. Otherwise, selecting the base dummy will become important
# use these variables ['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8'] to predict
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection
OzoneDroppedValues_Ozone=OzoneData.dropna(subset = ['Ozone','Solar.R'])
print OzoneDroppedValues_Ozone.count()
linreg = LinearRegression()

X = OzoneDroppedValues_Ozone[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]
y = OzoneDroppedValues_Ozone['Ozone']
linreg.fit(X,y)
pvals = feature_selection.f_regression(X,y)[1] #That's how we extract p-values
print(pvals)

Ozone      111
Solar.R    111
Wind       111
Temp       111
Month      111
Day        111
Month_5    111
Month_6    111
Month_7    111
Month_8    111
dtype: int64
[  1.79310857e-04   9.08941533e-13   1.55267723e-17   2.43194219e-03
   2.35681480e-01   2.51228945e-03   3.33469888e-03]


In [49]:
# now fill in null values of OzoneData['Ozone'] by predicted values
X = OzoneData[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]
X['Solar.R'].fillna(value = np.mean(X['Solar.R']), inplace = True)
print X.describe()
OzoneData['OZ_predict'] = linreg.predict(X)

OzoneData['Ozone'].fillna(value = OzoneData['OZ_predict'], inplace = True)
OzoneData.count()



          Solar.R        Wind        Temp     Month_5     Month_6     Month_7  \
count  151.000000  151.000000  151.000000  151.000000  151.000000  151.000000   
mean   185.931507    9.941722   78.165563    0.192053    0.198675    0.205298   
std     88.544727    3.524984    9.198138    0.395225    0.400331    0.405263   
min      7.000000    1.700000   57.000000    0.000000    0.000000    0.000000   
25%    119.000000    7.400000   73.000000    0.000000    0.000000    0.000000   
50%    197.000000    9.700000   79.000000    0.000000    0.000000    0.000000   
75%    257.000000   11.500000   85.000000    0.000000    0.000000    0.000000   
max    334.000000   20.700000   97.000000    1.000000    1.000000    1.000000   

          Month_8  
count  151.000000  
mean     0.205298  
std      0.405263  
min      0.000000  
25%      0.000000  
50%      0.000000  
75%      0.000000  
max      1.000000  


Ozone         151
Solar.R       146
Wind          151
Temp          151
Month         151
Day           151
Month_5       151
Month_6       151
Month_7       151
Month_8       151
OZ_predict    151
dtype: int64

In [50]:
# Now repeat previous steps for Solar.R variable using ['Ozone','Wind','Temp]
DroppedValues=OzoneData.dropna(subset = ['Solar.R'])
print DroppedValues.count()
linreg = LinearRegression()

X = DroppedValues[['Ozone','Temp']]
y = DroppedValues['Solar.R']
linreg.fit(X,y)

#Check p-values of your model - if a variable is not significant, drop it
pvals = feature_selection.f_regression(X,y)[1] #That's how we extract p-values
print(pvals)
### We shall drop Wind and rerun our regressions again


Ozone         146
Solar.R       146
Wind          146
Temp          146
Month         146
Day           146
Month_5       146
Month_6       146
Month_7       146
Month_8       146
OZ_predict    146
dtype: int64
[  4.79793585e-05   7.51772924e-04]


In [52]:
# now fill in null values of OzoneData['Solar.R'] by predicted values
X = OzoneData[['Ozone','Temp']]
OzoneData['SR_predict'] = linreg.predict(X)

OzoneData['Solar.R'].fillna(value = OzoneData['SR_predict'], inplace = True)
OzoneData.count()

Ozone         151
Solar.R       151
Wind          151
Temp          151
Month         151
Day           151
Month_5       151
Month_6       151
Month_7       151
Month_8       151
OZ_predict    151
SR_predict    151
dtype: int64

#### Now check your filled data - if your predicted values are more than maximum or less than minimum, replace them by max and min

In [56]:

OZ = [OzoneDroppedValues_Ozone['Ozone'].min(),OzoneDroppedValues_Ozone['Ozone'].max()]
SR = [DroppedValues['Solar.R'].min(),DroppedValues['Solar.R'].max()]

print OZ, SR

for i in range(0,(len(OzoneData))):
    if (OzoneData.loc[i,'Ozone'] < OZ[0]):
        OzoneData.loc[i,'Ozone'] = OZ[0]
print(OzoneData['Ozone'].min())


# we don't want to predict above max
for i in range(0,(len(OzoneData))):
    if (OzoneData.loc[i,'Ozone'] > OZ[1]):
        OzoneData.loc[i,'Ozone'] = OZ[1]
print(OzoneData['Ozone'].max())
    




[1.0, 168.0] [7.0, 334.0]


KeyError: 'the label [4] is not in the [index]'

## Bonus: In original question, replace missing values of Ozone and Solar.R by Single Imputation Regression Line with Error

In [ ]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)
OzoneData.dropna(how = 'all', subset = ['Ozone', 'Solar.R'], inplace = True)





